![banner](https://github.com/priyammaz/PyTorch-Adventures/blob/main/src/visuals/rl_banner.png?raw=true)

# Value Iteration 

In the [previous section](https://github.com/priyammaz/PyTorch-Adventures/blob/main/PyTorch%20for%20Reinforcement%20Learning/Intro%20to%20Reinforcement%20Learning/Model-Based%20Learning/intro_rl_and_policy_iter.ipynb), we dug deep into the basics of RL and first derived the famous ```Bellman Equation```

$$\text{Bellman Equation: } V_\pi(s) = \sum_a \pi(a|s)\sum_{s'}\sum_{r}[r + \gamma V_{\pi}(s')]P(s',r|a,s)$$

We then saw that we can manipulate and reinterpret the Bellman Equation a bit to create the ```Bellman Optimality Equations``` which provide a recursive form to solve our $Q$ and Values functions.

$$V^*(s) = \max_a \sum_{s'}\sum_{r}[r + \gamma V^*_{\pi}(s')]P(s',r|a,s)$$
$$Q^*(s,a) = \sum_{s'}\sum_{r}[r + \gamma \max_{a'}Q^*_\pi(s',a')]P(s',r|a,s)$$

But then we saw that to solve these equations could require an incredible large system of equations, making it intractable:

$$\boldsymbol{V_\pi} = (\boldsymbol{I} - \gamma\boldsymbol{P_\pi})^{-1}\boldsymbol{R_\pi}$$

Therefore, we have to use iterative methods to solve these systems. Previously we looked at the ```Policy Iteration``` method which had two steps:

$$\text{Policy Evaluation:  } V_{k+1}(s) = \sum_a \pi(a|s)\sum_{s'}\sum_{r}[r + \gamma V_k(s')]P(s',r|a,s)$$
$$\text{Policy Improvement:  } \pi'(s) = arg\max_a \sum_{s'}\sum_{r}[r + \gamma V_{\pi}(s')]P(s',r|a,s) = arg\max_a Q_\pi(s,a)$$

The intuition for Policy Iteration was, start with a random policy, iteratively compute its Value Function. Using the Value function, we can then compute our $Q$ value for every action at every state, and we select the action with the highest $Q$ Value to update our Policy. By repeating this, we will continue to improve our policy, but there is a catch: we have to also store the intermediate policy values. What about a different approach?

### Value Iteration

In Value Iteration, we don't care about the policies at all! All we want to do is, for our environment, find the **OPTIMAL VALUE FUNCTION**. The main difference between Value iteration here and the Policy Evaluation step in Policy Iteration is, in Policy Evaluation, we have a set Policy and we want to find the optimal Value function for that. Instead now, we want to find the optimal value function without fixing any policy at all. 

If we can update the Value function until convergence using our Bellman Equation, then at the end we can derive our Policy from the Optimal Values. 

$$\text{Value Iteration:  } V_{k+1}(s) = \max_a \sum_{s'}\sum_{r}[r + \gamma V_k(s')]P(s',r|a,s) = \max_a Q(s,a)$$

This should look very similar to our Policy Evaluation step except for one key difference: Instead of summing over the actions dictated by our policy, we simply update our Values with the action that provides the highest returns.

Therefore, this is basically an iterative method of solving for the Optimal Values Equation $V^*(s)$!

#### Strategy

At every state, compute the $Q$ value for every action, and pick the value for whichever action is the highest.

#### Implementation

There isn't much more to talk about, so lets go ahead and implement this on the Frozen Lake Game again! Also, this time, I won't do a separate stochastic vs deterministic implementation, this will work for both given the MDP! If you have any concerns though, start with the previous tutorial again!

In [1]:
import numpy as np
import gymnasium as gym

### Define Environment ###
env = gym.make('FrozenLake-v1', map_name="4x4", render_mode="rgb_array", is_slippery=True)

### Quick Helper Function to Nicely Print Policy ###
def print_policy(policy, grid=(4,4)):
    action_dict = {0: "Left", 1: "Down", 2: "Right", 3: "Up"}
    policy_print = np.empty(grid).astype(str)
    for idx_h in range(grid[0]):
        for idx_w in range(grid[1]):
            index = idx_h * grid[0] + idx_w
            selected_action = action_dict[policy[index]]
            selected_action = selected_action[0] # Grab first letter
            policy_print[idx_h, idx_w] = selected_action

    print("Current Policy:")
    print("--------------")
    print(policy_print)

In [2]:
def value_iteration(env, gamma=0.99, num_iterations=1000, tol=1e-5):

    ### Just Like Before, Initialize Values as 0 ###
    V = np.zeros(env.observation_space.n)

    for _ in range(num_iterations):

        ### Create a Copy of the Current Values Table ###
        V_k = np.copy(V)

        ### Loop Through Every State ###
        for s in range(env.observation_space.n):

            ### Create a List to Store Q Values in for every action for this state ###
            Q_s = []

            ### Loop Through The Action we Want to Take ###
            for wanted_action in range(env.action_space.n):

                ### List of Possible Destinations ###
                possible_taken_actions = env.unwrapped.P[s][wanted_action]

                ### Create a Q Value to Accumulate Into ###
                Q_sa = 0
                
                ### Loop Through Potential States We Could End Up In ###
                for probability, s_next, reward, terminal in possible_taken_actions:
                
                    ### Compute the Q Value (for this taken action) and Accumulate ###
                    Q_sa += probability * (reward + gamma * V_k[s_next])

                ### Store the Q Value for this action in our list ###
                Q_s.append(Q_sa)

            ### Whichever action gave the highest return, use that value in our updated Value Function ###
            V[s] = np.max(Q_s)

        ### Check for convergence
        if np.max(np.abs(V - V_k)) < tol:
            break

    return V
        
optimal_values = value_iteration(env)

print("Optimal Values")
print(np.array(optimal_values).reshape(4,4))

Optimal Values
[[0.54185998 0.49858161 0.47043461 0.45657012]
 [0.55829709 0.         0.35822941 0.        ]
 [0.59166815 0.64298202 0.6151213  0.        ]
 [0.         0.74165099 0.86280139 0.        ]]


## Compute our Policy

Now that we have our optimal Values function, lets use it to compute our Policy! But wait havent we dont that already? 

In Policy Iteration we iterate between two steps, the first to compute our Values, and the second to get the best Policy from those values. If thats the case, I have the best values already, so why not use the ```Policy Improvement``` code to extract the best policy?

$$\pi'(s) = arg\max_a \sum_{s'}\sum_{r}[r + \gamma V^*(s')]P(s',r|a,s) = arg\max_a Q(s,a)$$

The key difference here is, in the Policy Improvement step, we just had some $V_\pi(s')$ which were the optimal values for **THAT SPECIFIC POLICY**. Instead we can now use $V^*(s')$ because we have calculated our optimal values for the game.


In [3]:
def policy_improvement(values, gamma=0.99):

    new_policy = np.zeros(env.observation_space.n)

    for s in range(env.observation_space.n):

        Q_s = []

        ### For a state, iterate through all the actions I could take
        for wanted_action in range(env.action_space.n):

            ### Although I wanted to take that action, I could end up somewhere else, 
            ### This is a list of tuples as outlined earlier of potential destinations
            possible_taken_actions = env.unwrapped.P[s][wanted_action]

            ### Create an intermediate variable to accumulate into ###
            Q_sa = 0
            
            for probability, s_next, reward, terminal in possible_taken_actions:
                
                ### Compute the Q Value (for this taken action) and Accumulate ###
                Q_sa += probability * (reward + gamma * values[s_next])

            ### Store the Q Value (for this ###
            Q_s.append(Q_sa)

        ### Find Which Action had the largest Q ###
        best_action = np.argmax(Q_s) 

        ### Update Policy (for this state) with the new best action ###
        new_policy[s] = best_action

    return new_policy

optimal_policy = policy_improvement(optimal_values)
print_policy(optimal_policy)


Current Policy:
--------------
[['L' 'U' 'U' 'U']
 ['L' 'L' 'L' 'L']
 ['U' 'D' 'L' 'L']
 ['L' 'R' 'D' 'L']]


### Lets Use Our Policy 

Finally, lets put our policy to work and play the game with it. Just like before, due to randomness, even if our policy is optimal, we may still loose. So lets play a ton of games and see what proportion we win!

In [4]:
num_games = 1000
max_steps = 500

game_success = 0
for _ in range(num_games):

    observation, _ = env.reset()
    
    for _ in range(max_steps):
    
        # Select Action from Policy
        action = int(optimal_policy[observation])
        
        # Take the action and update the environment state
        observation, reward, done, _, _ = env.step(action)

        # Check if the game has ended
        if done:
            if reward > 0:
                game_success += 1
            break

proportion_sucessful = game_success / num_games
print("Proportion of Successful Games:", proportion_sucessful)

Proportion of Successful Games: 0.828
